# Tiktokshop

In [32]:
def summarize_api_overview(df):
    """
    Trả về tổng quan về API: số category, số scope, tổng số API
    """
    total_categories = df['category'].nunique()
    total_scopes = df['scope'].nunique()
    total_apis = len(df)
    
    return {
        "total_categories": total_categories,
        "total_scopes": total_scopes,
        "total_apis": total_apis
    }

In [33]:
import json
import pandas as pd

with open("./Document/tiktokapi_summary.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

df = df[df["scope"].notna()]                     # loại bỏ None
df = df[df["scope"].str.strip() != ""]           # loại bỏ chuỗi rỗng hoặc toàn khoảng trắng

df = df.reset_index(drop=True)

print(f"✅ Số API hợp lệ sau khi loại bỏ scope rỗng: {len(df)}")
display(df.head())
summarize_api_overview(df)


✅ Số API hợp lệ sau khi loại bỏ scope rỗng: 209


,title,description,scope,method,endpoint,category,use_case
0,Get Authorized Category Assets,Retrieves the list of business category assets...,partner.authorization.info,GET,/authorization/202405/category_assets,Authorization,
1,Get Authorized Shops,Retrieves the list of shops that a seller has ...,seller.authorization.info,GET,/authorization/202309/shops,Authorization,
2,Get Shop Webhooks,Retrieves a shop's webhooks and the correspond...,seller.authorization.info,GET,/event/202309/webhooks,Event,
3,Update Shop Webhook,Updates the shop's webhook URL for a specific ...,seller.authorization.info,PUT,/event/202309/webhooks,Event,
4,Delete Shop Webhook,Deletes the shop's webhook URL for a specific ...,seller.authorization.info,DELETE,/event/202309/webhooks,Event,


{'total_categories': 18, 'total_scopes': 43, 'total_apis': 209}

In [11]:
import pandas as pd

# Số lượng scope duy nhất
unique_scopes = df['scope'].nunique()
print(f"📊 Tổng số scope duy nhất: {unique_scopes}\n")

# Bước 1: đếm số lần mỗi scope xuất hiện trong từng category
scope_category_counts = df.groupby(['scope', 'category']).size().reset_index(name='Count_in_category')

# Bước 2: tổng số API có scope đó
scope_total_counts = df.groupby('scope').size().reset_index(name='Total_scope_count')

# Bước 3: kết hợp, tính Count_not_in_category = Total_scope_count - Count_in_category
scope_category_stats = pd.merge(scope_category_counts, scope_total_counts, on='scope')
scope_category_stats['Count_not_in_category'] = scope_category_stats['Total_scope_count'] - scope_category_stats['Count_in_category']

# Bước 4: giữ các cột cần thiết
scope_category_stats = scope_category_stats[['scope', 'category', 'Count_in_category', 'Count_not_in_category']]

# === Hiển thị kết quả ===
display(scope_category_stats.sort_values(by='Count_in_category', ascending=False).reset_index(drop=True))


📊 Tổng số scope duy nhất: 43



,scope,category,Count_in_category,Count_not_in_category
0,seller.product.basic,Products,26,0
1,seller.fulfillment.basic,Fulfillment,13,1
2,data.shop_analytics.public.read,Analytics,11,0
3,seller.return_refund.basic,Return and refund,11,0
4,seller.affiliate_collaboration.read,Affiliate seller,10,0
5,seller.affiliate_collaboration.write,Affiliate seller,10,0
6,partner.tap_campaign.read,Affiliate partner,9,0
7,seller.customer_service,Customer service,9,0
8,creator.affiliate_collaboration.read,Affiliate creator,8,0
9,seller.product.write,Products,6,0


+ `Count_in_category`: số API có scope đó trong category đó
+ `Count_not_in_category`: số API có scope đó nhưng không thuộc category đó

Báo cáo tổng quan về API

Dưới đây là tổng quan về hệ thống API hiện tại:

| Mục | Số lượng |
|-----|----------|
| Tổng số **Categories** | 18 |
| Tổng số **Scopes** | 43 |
| Tổng số **APIs** | 209 |

## Tiktokshop API Collectors

In [34]:
import json
import pandas as pd

with open("./Document/api_collect_results.json", "r", encoding="utf-8") as f:
    data = json.load(f)

dfTiktok = pd.DataFrame(data)

dfTiktok = dfTiktok.reset_index(drop=True)

print(f"✅ Số API hợp lệ sau khi loại bỏ scope rỗng: {len(dfTiktok)}")
display(dfTiktok.head())
summarize_api_overview(dfTiktok)

✅ Số API hợp lệ sau khi loại bỏ scope rỗng: 117


,title,description,category,scope,method,endpoint,use_case
0,Check Listing Prerequisites,Check if a TikTok shop is ready to list produc...,Products,seller.product.basic,GET,/product/202312/prerequisites,Check if a shop meets all requirements before ...
1,Get Categories,Retrieve the list of product categories availa...,Products,seller.product.basic,GET,/product/202309/categories,Retrieve the latest list of available product ...
2,Get Category Rules,Retrieve the additional requirements (beyond m...,Products,seller.product.basic,GET,/product/202309/categories/{category_id}/rules,Retrieve specific listing requirements and sup...
3,Get Attributes,Retrieve the standard built-in product and sal...,Products,seller.product.basic,GET,/product/202309/categories/{category_id}/attri...,Retrieve standard product and sales attributes...
4,Get Brands,"Retrieve all available brands for your shop, i...",Products,seller.product.basic,GET,/product/202309/brands,"Retrieve all available brands for the shop, in..."


{'total_categories': 15, 'total_scopes': 27, 'total_apis': 117}

Báo cáo tổng quan về API

Dưới đây là tổng quan về hệ thống API hiện tại:

| Mục | Số lượng |
|-----|----------|
| Tổng số **Categories** | 15 |
| Tổng số **Scopes** | 27 |
| Tổng số **APIs** | 117 |

In [25]:
import pandas as pd

# Số lượng scope duy nhất
unique_scopes = dfTiktok['scope'].nunique()
print(f"📊 Tổng số scope duy nhất: {unique_scopes}\n")

# Bước 1: đếm số lần mỗi scope xuất hiện trong từng category
scope_category_counts = dfTiktok.groupby(['scope', 'category']).size().reset_index(name='Count_in_category')

# Bước 2: tổng số API có scope đó
scope_total_counts = dfTiktok.groupby('scope').size().reset_index(name='Total_scope_count')

# Bước 3: kết hợp, tính Count_not_in_category = Total_scope_count - Count_in_category
scope_category_stats = pd.merge(scope_category_counts, scope_total_counts, on='scope')
scope_category_stats['Count_not_in_category'] = scope_category_stats['Total_scope_count'] - scope_category_stats['Count_in_category']

# Bước 4: giữ các cột cần thiết
scope_category_stats = scope_category_stats[['scope', 'category', 'Count_in_category', 'Count_not_in_category']]

# === Hiển thị kết quả ===
display(scope_category_stats.sort_values(by='Count_in_category', ascending=False).reset_index(drop=True))

📊 Tổng số scope duy nhất: 27



,scope,category,Count_in_category,Count_not_in_category
0,seller.product.basic,Products,14,0
1,data.shop_analytics.public.read,Analytics,11,0
2,seller.affiliate_collaboration.read,Affiliate seller,10,0
3,partner.tap_campaign.read,Affiliate partner,8,0
4,creator.affiliate_collaboration.read,Affiliate creator,8,0
5,seller.finance.info,Finance,6,0
6,seller.fbt.info,Fulfilled by TikTok (FBT),6,0
7,seller.return_refund.basic,Return and refund,5,0
8,seller.customer_service,Customer service,4,0
9,seller.global_product.category.info,Products,4,0


In [29]:
def get_apis_by_category(df, category_name):
    scopes = df[df['category'] == category_name]['scope'].unique().tolist()
    return scopes

# Ví dụ:
products_apis = get_apis_by_category(dfTiktok, 'Products')
orders_apis = get_apis_by_category(dfTiktok, 'Orders')

print("📦 APIs cho category Products:")
for api in products_apis:
    print("-", api)

print("\n📝 APIs cho category Orders:")
for api in orders_apis:
    print("-", api)


📦 APIs cho category Products:
- seller.product.basic
- seller.product.basic, serller.product.optimize
- serller.product.optimize
- seller.global_product.category.info
- seller.global_product.info

📝 APIs cho category Orders:
- seller.order.info
- seller.order.ext_ref.read


# Shopee